Here I will make a dictionary based sentiment analasys that looks at the data and decided whether or not the comment is positive or negative or somewhere in between

In [2]:
import pandas as pd
df = pd.read_csv("smallData.csv")
df['reviewText']= df['reviewText'].astype(str)
ratings = list(df["overall"])
reviews = list(df["reviewText"])

Below we are downloading the dictionary that has a comprehensive list of negative and positive words. We will use this dictionary to figure out whether a review is negative or positive

In [3]:
import nltk
from nltk.corpus import opinion_lexicon
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('opinion_lexicon') #Download Opinion Dictionary
positive_wds = set(opinion_lexicon.positive())
negative_wds = set(opinion_lexicon.negative())


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\navneeth\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


Below is the function sentence score that takes a sentence and looks through each word to see whether it is positive or negative. By looking at whether there are more positive or negative words, I am thinking that I can find the overall sentiment of the sentence (May be naive). For each sentence this below funciton would return a score of 1 or -1.

In [4]:
def sentenceScore(sentence):
    filtered_words = []
    lowercaseWords = []
    pos = 0
    neg = 0
    for word in sentence: #if alphanumeric, append word to list
        if(word.isalnum()):
            filtered_words.append(word)
    
    sentence = filtered_words
    
    sentence = filtered_words
    
    for word in sentence: #make all words lowercase
        lowercaseWords.append(word.lower())
    sentence = lowercaseWords

    if not sentence:
        return 0

    total = len(sentence)
    for word in sentence:
        if(word in positive_wds):
            pos += 1
        if(word in negative_wds):
            neg+=1
    
    return (pos-neg)/total

Below tests whether the above function works

In [5]:

sen = "Happy"
tokens = word_tokenize(sen)

print(sentenceScore(tokens))

1.0


Below is a function that makes use of the above function and returns the average sentiment scores of a sentence in a review

In [6]:
def score_review(review):
    sentimentScores = [] #list with scores for all sentences in a review
    sentences = sent_tokenize(review) #seperates a full review into multiple sentences
    for sentence in sentences:
        words = word_tokenize(sentence) #seperates a sentence into words
        sentimentScores.append(sentenceScore(words))
    return sum(sentimentScores)/len(sentimentScores) #returns average sentimentScores of a sentence in a review
    
     

Below puts everything together and runs the program. We write to a file named dictBasedSentiments.csv

In [7]:
reviewSentiments = []
for review in reviews:
    reviewSentiments.append(score_review(review))

df = pd.DataFrame({
        "rating": ratings,
        "review": reviews,
        "review dictionary based sentiment": reviewSentiments,
})
df.to_csv('dictBasedSentiment.csv')

Now we have to evaluate how well this program works in evaluating the sentiment of customer reviews by comparing it to the overall review. We will normalize both the prediction data and the customer review data and then compare.

In [8]:
def score_to_rating(value): #this normalizes the dictbasedsentiment data to a score of 0 for negative, 1 for neutral and 2 for positive
    if value > 0.2:
        return 2
    if value <= 0.2 and value >= -0.2:
        return 1
    else:
        return 0
df['predicted'] = df['review dictionary based sentiment'].apply(lambda x:score_to_rating(x))

In [9]:
def score_to_Target(value): #this normalizes the actual review rating data
    if value >= 5:
        return 2
    if value <= 4 and value >= 2:
        return 1
    else:
        return 0
df['target'] = df['rating'].apply(lambda x:score_to_Target(x))

Below is the code to run all this and make a table evaluating prevision. We can see that there is a clear bias towards the neutral as the recall is so high. 
The low accuracy shows that this is not that precise of a model

In [10]:
rating_classes = list(df["target"])
sentiment_values  = list(df["predicted"])
from sklearn.metrics import classification_report
target_names = ["negative", "neutral", "positive"]
print(classification_report(rating_classes, sentiment_values, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.82      0.05      0.10      1500
     neutral       0.36      0.93      0.52      1500
    positive       0.78      0.28      0.41      1500

    accuracy                           0.42      4500
   macro avg       0.66      0.42      0.34      4500
weighted avg       0.66      0.42      0.34      4500

